In [240]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import inspect
from config import password

# Participant Data

### Extract

In [241]:
# File paths to datasets
## Current ##
# Current dataset obtained at https://data.ndis.gov.au/explore-data, filter States/Territories = WA
# File name contains today's date
today = '2022-09-27'
#today = datetime.today().strftime('%Y-%m-%d')
filename = f"data-export-participants-{today}.csv"
part = f"resources/{filename}"
wa_curr_part_df = pd.read_csv(part)
wa_curr_part_df.head()

,StateCd,SrvcDstrctNm,DsbltyGrpNm,AgeBnd,PrtcpntCnt,AvgAnlsdCmtdSuppBdgt,SuppCatg
0,WA,ALL,ABI,0 to 6,13,"105,000.00",ALL
1,WA,ALL,ABI,0 to 6,13,"40,000.00",Capacity Building
2,WA,ALL,ABI,0 to 6,11,0,Capital
3,WA,ALL,ABI,0 to 6,11,0,Core
4,WA,ALL,ABI,15 to 18,43,"91,000.00",ALL


In [242]:
## Longitudinal data
# Longitudinal data obtained at https://data.ndis.gov.au/data-downloads, under subheading 'Participant numbers and average plan budgets (September 2020 to June 2022)'
# !!! Note if this process is used beyond December 2022, additional longitudinal datasets will need to be added here manually
# eg. sep22, dec22, etc

jun22_df = pd.read_csv("resources/PB Participant numbers and plan budgets data JUN22_0.csv")
mar22_df = pd.read_csv("resources/PB Participant Numbers and Plan Budgets_MAR22_CSV.csv")
dec21_df = pd.read_csv("resources/PB Participant numbers and Plan Budgets data_DEC21_CSV.csv")
sep21_df = pd.read_csv("resources/PB Participant numbers and plan budgets data Sep2021 CSV.csv")
jun21_df = pd.read_csv("resources/PB Active Plan Participant Jun2021.csv")
mar21_df = pd.read_csv("resources/PB Active Plan Participant Mar 2021.csv")
dec20_df = pd.read_csv("resources/PB Active Plan Participant Dec 2020.csv")
sep20_df = pd.read_csv("resources/PB Active Plan Participant Sep 2020 CSV.csv")

### Transform

In [243]:
## Longitudinal data
# Make Date column format consistent

mar22_df.RprtDt = mar22_df.RprtDt.replace('31-Mar-22','31MAR2022')
dec21_df.RprtDt = dec21_df.RprtDt.replace('31-Dec-21','31DEC2021')
sep21_df.RprtDt = sep21_df.RprtDt.replace('30-Sep-21','30SEP2021')

In [244]:
# Append Longitudinal dataframes to a single dataframe
long_list = [jun22_df,mar22_df,dec21_df,sep21_df,jun21_df,mar21_df,dec20_df,sep20_df]

long_df = jun22_df
for i in range(1,8):
    long_df = long_df.append(long_list[i])

In [245]:
# Filter Longitudinal dataframe to WA only
wa_long_part_df = long_df.loc[long_df.StateCd == "WA", :]
wa_long_part_df

,RprtDt,StateCd,SrvcDstrctNm,DsbltyGrpNm,AgeBnd,SuppClass,AvgAnlsdCmtdSuppBdgt,PrtcpntCnt
82761,30JUN2022,WA,ALL,ABI,0 to 6,ALL,"105,000.00",13
82762,30JUN2022,WA,ALL,ABI,0 to 6,CapacityBuilding,"40,000.00",13
82763,30JUN2022,WA,ALL,ABI,0 to 6,Capital,NaN,<11
82764,30JUN2022,WA,ALL,ABI,0 to 6,Core,"53,000.00",11
82765,30JUN2022,WA,ALL,ABI,15 to 18,ALL,"91,000.00",43
...,...,...,...,...,...,...,...,...
88476,30SEP2020,WA,Wheat Belt,Visual Impairment,65+,Core,NaN,11
88477,30SEP2020,WA,Wheat Belt,Visual Impairment,ALL,ALL,NaN,11
88478,30SEP2020,WA,Wheat Belt,Visual Impairment,ALL,CapacityBuilding,NaN,11
88479,30SEP2020,WA,Wheat Belt,Visual Impairment,ALL,Capital,NaN,11


In [246]:
# Transform wa_df to match wa_curr_part_df
# Change column name
wa_long_part_df = wa_long_part_df.rename(columns={'SuppClass':'SuppCatg'})

# Change value for consistency
wa_long_part_df["SuppCatg"] = wa_long_part_df["SuppCatg"].replace("CapacityBuilding", "Capacity Building")

In [247]:
# Add date to wa_curr_part_df
# !!! Note if this process is used beyond December 2022, the date entered will need to be updated
# eg. '31DEC2022', '31MAR2022', etc
wa_curr_part_df["RprtDt"] = '30SEP2022'

# Append the current df to the longitudinal df
wa_part_df = wa_curr_part_df.append(wa_long_part_df)

# Drop the state code and annual support budget column
wa_part_df.drop(["StateCd", "AvgAnlsdCmtdSuppBdgt"], axis=1, inplace=True)

# In Participant Count <11 with 5 (See note in ReadMe)
wa_part_df["PrtcpntCnt"] = wa_part_df["PrtcpntCnt"].replace("<11","5", regex=True)

In [248]:
# Set data types
wa_part_df = wa_part_df.astype({'SrvcDstrctNm': 'string',
                                    'DsbltyGrpNm': 'string',
                                    'AgeBnd': 'string',
                                    'PrtcpntCnt': 'int64',
                                    'SuppCatg': 'string',
                                    'RprtDt': 'string'})

wa_part_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60051 entries, 0 to 88480
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SrvcDstrctNm  60051 non-null  string
 1   DsbltyGrpNm   60051 non-null  string
 2   AgeBnd        60051 non-null  string
 3   PrtcpntCnt    60051 non-null  int64 
 4   SuppCatg      60051 non-null  string
 5   RprtDt        60051 non-null  string
dtypes: int64(1), string(5)
memory usage: 3.2 MB


# Plan Utilisation Data

### Extract

In [249]:
# File paths to datasets
## Current ##
# Current dataset obtained at https://data.ndis.gov.au/explore-data, filter States/Territories = WA
# File name contains today's date
today = '2022-09-27'
#today = datetime.today().strftime('%Y-%m-%d')
filename = f"data-export-utilisation-plan-budget-{today}.csv"
util = f"resources/{filename}"
wa_curr_util_df = pd.read_csv(util)
wa_curr_util_df.head()

,StateCd,SrvcDstrctNm,DsbltyGrpNm,AgeBnd,SuppClass,TtlCmtd,Utlstn,PrtcpntCnt,PrvdrCnt
0,WA,ALL,ABI,0 to 6,ALL,"87,000.00",75%,12,47
1,WA,ALL,ABI,0 to 6,Capacity Building,"45,000.00",66%,12,26
2,WA,ALL,ABI,15 to 18,ALL,"95,000.00",84%,42,98
3,WA,ALL,ABI,15 to 18,Capacity Building,"28,000.00",66%,42,64
4,WA,ALL,ABI,15 to 18,Capital,"18,000.00",40%,20,22


In [250]:
## Longitudinal data
# Longitudinal data obtained at https://data.ndis.gov.au/data-downloads, under subheading 'Budget data downloads (September 2020 to June 2022)'
# !!! Note if this process is used beyond December 2022, additional longitudinal datasets will need to be added here manually
# eg. sep22, dec22, etc
jun22_df = pd.read_csv("resources/PB Utilisation of plan budgets data JUN22.csv")
mar22_df = pd.read_csv("resources/PB Utilisation CSV MAR22_CSV.csv")
dec21_df = pd.read_csv("resources/PB Utilisation of Plan Budgets_DEC21_CSV.csv")
sep21_df = pd.read_csv("resources/PB Utilisation of Plan Budgets Sep21 CSV.csv")
jun21_df = pd.read_csv("resources/PB Utilisation of Plan Budgets Jun 2021 CSV.csv")
mar21_df = pd.read_csv("resources/PB Utilisation of Plan Budgets Mar 2021.csv")
dec20_df = pd.read_csv("resources/PB Utilisation of Plan Budgets Dec 2020.csv")
sep20_df = pd.read_csv("resources/PB Utilisation of Plan Budgets Sep 2020 CSV.csv")

### Transform

In [251]:
## Longitudinal data
# Make Date column format consistent
mar22_df.RprtDt = mar22_df.RprtDt.replace('31-Mar-22','31MAR2022')
dec21_df.RprtDt = dec21_df.RprtDt.replace('31-Dec-21','31DEC2021')
sep21_df.RprtDt = sep21_df.RprtDt.replace('30-Sep-21','30SEP2021')
jun21_df.RprtDt = jun21_df.RprtDt.replace('30Jun2021','30JUN2021')

In [252]:
# Make column headings consistent
jun22_df = jun22_df.rename(columns={"suppclass":"SuppClass"})
mar22_df = mar22_df.rename(columns={"Suppclass":"SuppClass"})

In [253]:
# Append Longitudinal dataframes to a single dataframe
long_list = [jun22_df,mar22_df,dec21_df,sep21_df,jun21_df,mar21_df,dec20_df,sep20_df]

long_df = jun22_df
for i in range(1,8):
    long_df = long_df.append(long_list[i])

In [254]:
# Filter Longitudinal dataframe to WA only
wa_long_util_df = long_df.loc[long_df.StateCd == "WA", :]

In [255]:
# Add date to wa_curr_part_df
# !!! Note if this process is used beyond December 2022, the date entered will need to be updated
# eg. '31DEC2022', '31MAR2022', etc
wa_curr_util_df["RprtDt"] = '30SEP2022'

# Append the current df to the longitudinal df
wa_util_df = wa_curr_util_df.append(wa_long_util_df)

# Drop the state code and other unnecessary columns
wa_util_df.drop(["StateCd","PrtcpntCnt","PrvdrCnt","SILorSDA"], axis=1, inplace=True)

# Rename column for consistency
wa_util_df = wa_util_df.rename(columns={"SuppClass":"SuppCatg"})

# Drop rows with missing data
wa_util_df = wa_util_df[wa_util_df["SuppCatg"] != "Missing"]

In [256]:
# Clean punctuation and convert blank spaces to NaN
wa_util_df["TtlCmtd"] = wa_util_df["TtlCmtd"].replace(",","", regex=True)
wa_util_df["TtlCmtd"] = wa_util_df["TtlCmtd"].replace(".00","", regex=True)
wa_util_df["TtlCmtd"] = wa_util_df["TtlCmtd"].replace(r"^\s*$", np.nan, regex=True)
wa_util_df["Utlstn"] = wa_util_df["Utlstn"].replace("%","", regex=True)

In [257]:
# Set data types
wa_util_df = wa_util_df.astype({'SrvcDstrctNm': 'string',
                                    'DsbltyGrpNm': 'string',
                                    'AgeBnd': 'string',
                                    'SuppCatg': 'string',
                                    'RprtDt': 'string',
                                    'TtlCmtd': 'float64',
                                    'Utlstn': 'int64'})

wa_util_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70374 entries, 0 to 115330
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SrvcDstrctNm  70374 non-null  string 
 1   DsbltyGrpNm   70374 non-null  string 
 2   AgeBnd        70374 non-null  string 
 3   SuppCatg      70374 non-null  string 
 4   TtlCmtd       3954 non-null   float64
 5   Utlstn        70374 non-null  int64  
 6   RprtDt        70374 non-null  string 
dtypes: float64(1), int64(1), string(5)
memory usage: 4.3 MB


# Composite Key

In [258]:
# Create a table that generates a composite key from the following columns:
# SrvcDstrctNm	DsbltyGrpNm	AgeBnd	SuppCatg	RprtDt
# This key will allow the participant dataframe and utilisation dataframes to be joined (in a RDBMS)
# !!! Note if this process is used beyond December 2022, add additional values to *end* of the date list
# eg '31DEC2022', '31MAR2023', etc

dates = ['30SEP2020', '31DEC2020', '31MAR2021', '30JUN2021', '30SEP2021', '31DEC2021', '31MAR2022',
 '30JUN2022', '30SEP2022']
districts = wa_part_df.SrvcDstrctNm.unique()
disability_groups = wa_part_df.DsbltyGrpNm.unique()
age_bands = wa_part_df.AgeBnd.unique()
support_types = wa_part_df.SuppCatg.unique()

key_categories = []

for date in dates:
    for district in districts:
        for disability_group in disability_groups:
            for age_band in age_bands:
                for support_type in support_types:
                    key_categories.append((date,district,disability_group,age_band,support_type))


In [259]:
# Create key dataframe
key_df = pd.DataFrame(key_categories)
key_df["concatenate"] = key_df[[0,1,2,3,4]].apply("-".join, axis=1)
key_df["Key"] = key_df.index + 1
key_df = key_df[["Key", "concatenate"]]
key_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91800 entries, 0 to 91799
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Key          91800 non-null  int64 
 1   concatenate  91800 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [260]:
# Add concatenate column to wa_part_df and wa_util_df
wa_part_df["concatenate"] = wa_part_df[["RprtDt","SrvcDstrctNm","DsbltyGrpNm","AgeBnd","SuppCatg"]].apply("-".join, axis=1)
wa_util_df["concatenate"] = wa_util_df[["RprtDt","SrvcDstrctNm","DsbltyGrpNm","AgeBnd","SuppCatg"]].apply("-".join, axis=1)

wa_part_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60051 entries, 0 to 88480
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SrvcDstrctNm  60051 non-null  string
 1   DsbltyGrpNm   60051 non-null  string
 2   AgeBnd        60051 non-null  string
 3   PrtcpntCnt    60051 non-null  int64 
 4   SuppCatg      60051 non-null  string
 5   RprtDt        60051 non-null  string
 6   concatenate   60051 non-null  object
dtypes: int64(1), object(1), string(5)
memory usage: 3.7+ MB


In [261]:
# Merge wa_part_df and wa_util_df to key_df on concatenate column
wa_part_df = pd.merge(wa_part_df, key_df, on="concatenate", how="left")
wa_util_df = pd.merge(wa_util_df, key_df, on="concatenate", how="left")

# Drop concatenate column in both wa_part_df and wa_util_df
wa_part_df.drop("concatenate", axis=1, inplace=True)
wa_util_df.drop("concatenate", axis=1, inplace=True)

# Load

In [262]:
# The load component for this project is performed in pgAdmin4 (or an equivalent SQL management tool)
# See the ReadMe for more details

# Export dataframes to CSV
wa_part_df.to_csv('output/participant_data.csv', index=False)
wa_util_df.to_csv('output/plan_utlisation_data.csv', index=False)